In [0]:
!pip install tensorflow==2.0.0

In [0]:
# Imports
import os 
import csv 

import numpy as np 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

In [0]:
uploaded=files.upload()

In [5]:
# Data Loading

def get_data(filename):
    with open(filename) as training_file:
        csv_reader = csv.reader(training_file, delimiter=',')
        first_line = True
        temp_images = []
        temp_labels = []
        for row in csv_reader:
            if first_line:
                print("Ignoring First Line")
                first_line = False
            else:
                temp_labels.append(row[0])
                image_data = row[1:785]
                image_data_as_array = np.array_split(image_data, 28)
                temp_images.append(image_data_as_array)
        images = np.array(temp_images).astype('float')
        labels = np.array(temp_labels).astype('float')
    return images, labels

train_images, train_labels = get_data('sign_mnist_train.csv')
test_images, test_labels = get_data('sign_mnist_test.csv')

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

Ignoring First Line
Ignoring First Line
(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [10]:
# Prepping Data 
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255.)

print(train_images.shape)
print(test_images.shape)

(27455, 28, 28, 1, 1)
(7172, 28, 28, 1, 1)


In [7]:
# Training
from tensorflow.keras.optimizers import Adam

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(28, activation='softmax')
])

model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit_generator(train_datagen.flow(train_images, train_labels, batch_size=32), 
                              validation_data=validation_datagen.flow(test_images, test_labels, batch_size=32),
                              epochs=15,
                              steps_per_epoch=len(train_images) / 32,
                              validation_steps=len(test_images) / 32,
                              verbose=2)

model.evaluate(test_images, test_labels)

Epoch 1/15
858/857 - 79s - loss: 2.9657 - accuracy: 0.1099 - val_loss: 2.3654 - val_accuracy: 0.2646
Epoch 2/15
858/857 - 79s - loss: 2.2607 - accuracy: 0.2835 - val_loss: 1.5072 - val_accuracy: 0.4787
Epoch 3/15
858/857 - 79s - loss: 1.9110 - accuracy: 0.3843 - val_loss: 1.3480 - val_accuracy: 0.5374
Epoch 4/15
858/857 - 79s - loss: 1.7622 - accuracy: 0.4263 - val_loss: 1.2927 - val_accuracy: 0.5697
Epoch 5/15
858/857 - 78s - loss: 1.6269 - accuracy: 0.4651 - val_loss: 1.2496 - val_accuracy: 0.5719
Epoch 6/15
858/857 - 79s - loss: 1.5285 - accuracy: 0.4961 - val_loss: 1.0155 - val_accuracy: 0.6340
Epoch 7/15
858/857 - 79s - loss: 1.4231 - accuracy: 0.5263 - val_loss: 0.9594 - val_accuracy: 0.6716
Epoch 8/15
858/857 - 78s - loss: 1.3487 - accuracy: 0.5520 - val_loss: 0.9305 - val_accuracy: 0.6725
Epoch 9/15
858/857 - 79s - loss: 1.2615 - accuracy: 0.5818 - val_loss: 0.9063 - val_accuracy: 0.6810
Epoch 10/15
858/857 - 79s - loss: 1.2108 - accuracy: 0.5941 - val_loss: 0.8922 - val_accura

[419.7261569719373, 0.3575014]

In [0]:
# Plotting
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.grid()
plt.legend()
plt.figure()

plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation loss')
plt.grid()
plt.legend()
plt.show()